## UnderStat is a website detailing results and stats from top tier football matches

The purpose of this notebook is to scrape Understat of all data related to final league positions and match statistics from the past 7 years for the top 5 leagues in world football

In [6]:
#packages used
import numpy as np
import pandas as pd 
import requests 
from bs4 import BeautifulSoup as bs 
import ast 
import json
from selenium import webdriver
import time

There are two methods we can use to get league positions. The first is to scrape the browser-shown data from each league as below


In [21]:
driver = webdriver.Chrome()
sample_url = "https://understat.com/league/EPL/2018"

driver.get(sample_url)
time.sleep(5)
#scrape the 'table'
table_outline = driver.find_element_by_xpath('//div[contains(@id, "league-chemp")]')
table = table_outline.find_element_by_tag_name('table')
print(table.text)


№ Team M W D L G GA PTS xG xGA xPTS
1 Manchester City 38 32 2 4 95 23 98 93.72-1.28 25.73+2.73 90.64-7.36
2 Liverpool 38 30 7 1 89 22 97 79.46-9.54 29.15+7.15 83.45-13.55
3 Chelsea 38 21 9 8 63 39 72 63.97+0.97 38.11-0.89 71.45-0.55
4 Tottenham 38 23 2 13 67 39 71 61.75-5.25 49.15+10.15 61.44-9.56
5 Arsenal 38 21 7 10 73 51 70 64.80-8.20 57.30+6.30 58.97-11.03
6 Manchester United 38 19 9 10 65 54 66 68.62+3.62 52.30-1.70 61.86-4.14
7 Wolverhampton Wanderers 38 16 9 13 47 46 57 53.06+6.06 42.69-3.31 59.91+2.91
8 Everton 38 15 9 14 54 46 54 53.87-0.13 49.31+3.31 55.54+1.54
9 Leicester 38 15 7 16 51 48 52 52.11+1.11 44.64-3.36 56.24+4.24
10 West Ham 38 15 7 16 52 55 52 47.96-4.04 65.66+10.66 43.72-8.28
11 Watford 38 14 8 16 52 59 50 51.73-0.27 63.29+4.29 46.00-4.00
12 Crystal Palace 38 14 7 17 51 53 49 50.64-0.36 52.80-0.20 51.93+2.93
13 Newcastle United 38 12 9 17 42 48 45 39.91-2.09 57.55+9.55 39.22-5.78
14 Bournemouth 38 13 6 19 56 70 45 58.81+2.81 62.42-7.58 51.49+6.49
15 Burnley 38 1

We could then split up each line (sep = '\n') and each stat (sep = ' ') and iterate across all the leagues.

However, because we want to also get the individual match stats we will not use this method, instead looking at the data behind the site in scrips